In [601]:
import pandas as pd
import numpy as np
import math
from datetime import date, time, datetime, timedelta
import subprocess
import bandit.main as bd
import calendar

In [602]:
def set_cohorts(source_df, score_col):
    cohort_size = 3
    cohort_size_2 = cohort_size**2
    cohort_size_3 = cohort_size**3

    # break out distinct scores
    uniq_list = source_df[score_col].unique()
    df = pd.DataFrame({score_col : uniq_list}).sort_values(score_col, ascending=True)
    df = df.sort_values (score_col)
    df = df.reset_index (drop=True)
    
    if len (df) > cohort_size:
        grp_size = len (df) / cohort_size
        df['cohort1'] = df.index // grp_size

    if len (df) > cohort_size_2:
        grp_size = len (df) / cohort_size_2
        df['cohort2'] = df.index // grp_size
        
    if len (df) > cohort_size_3:
        grp_size = len (df) / cohort_size_3
        df['cohort3'] = df.index // grp_size

    return df

In [603]:
def df_search(df_in, cohort):
    if not cohort in df_in:
        return df_in

    df_class = df_bandit_class(df_in, cohort)
    print(df_class)
    next_class = df_class.iloc[0, 0]
    if df_in['Complete'].isnull().sum() > 0:
        df_filtered = df_in[df_in[cohort] == next_class]
    else:
        df_filtered = df_in
        
    return df_filtered

In [604]:
def set_start(df):
    goal_minutes = df['Goal']
    final = datetime.combine(date.today(), time(0, 0)) + timedelta(minutes=goal_minutes)
    return final

In [605]:
# todo: combine bandit_class and bandit_diff
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    
    # set up the time
    # dfgb['CanStart'] = dfgb.apply(set_start, axis=1)
    
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [606]:
# timelist = [datetime.combine(date.today(), time(0, 0)) + timedelta(minutes=i) for i in  range(0,1440,5)]
minute_list = [i for i in  range(5,1440,5)]
df = pd.DataFrame({"Start":minute_list})

In [607]:
currDate = datetime.today()
minute_list = [i for i in  range(5,1440,5)]
df = pd.DataFrame({"Goal":minute_list})
df["Date"] = currDate
df["Complete"] = np.nan
df = df[['Date', 'Goal', 'Complete']]

In [608]:
subprocess.Popen('StopSmoking.xlsx', shell=True)

In [609]:
df_excel = pd.read_excel(io='StopSmoking.xlsx')
df = df.append(df_excel).reset_index(drop=True)

In [610]:
df = bd.reduce(df,'Goal')

1 0.9975 1


In [611]:
df.tail(10)

Date  Goal  Complete
278 2018-10-06 02:13:33.213988  1395       NaN
279 2018-10-06 02:13:33.213988  1400       NaN
280 2018-10-06 02:13:33.213988  1405       NaN
281 2018-10-06 02:13:33.213988  1410       NaN
282 2018-10-06 02:13:33.213988  1415       NaN
283 2018-10-06 02:13:33.213988  1420       NaN
284 2018-10-06 02:13:33.213988  1425       NaN
285 2018-10-06 02:13:33.213988  1430       NaN
286 2018-10-06 02:13:33.213988  1435       NaN
287 2018-10-06 00:00:00.000000     5       1.0

In [612]:
df['Score'] = np.where(df['Complete'] >= 1, df['Goal']**1.2, 0)
df.loc[df['Complete'].isnull(), 'Score'] = np.NaN

In [613]:
df_quality = bd.get_quality(df['Score'])

In [614]:
df = df.join(df_quality).sort_values('Quality', ascending=False)

In [615]:
val_df = set_cohorts(df, 'Goal')
df = df.merge(val_df, on='Goal')

In [616]:
currDay = calendar.day_name[currDate.weekday()]
df_today = df
df_today['weekday'] = df_today['Date'].dt.day_name()
df_today = df_today[df_today['weekday'] == currDay]
today_sum = df_today['Complete'].notnull().sum()
if today_sum >= 10:
    df = df_today

In [617]:
df.sort_values('Date', ascending=True).tail(10)

Date  Goal  Complete  Score  Quality  cohort1  \
98  2018-10-06 02:13:33.213988   490       NaN    NaN      NaN      1.0   
97  2018-10-06 02:13:33.213988   485       NaN    NaN      NaN      1.0   
96  2018-10-06 02:13:33.213988   480       NaN    NaN      NaN      0.0   
95  2018-10-06 02:13:33.213988   475       NaN    NaN      NaN      0.0   
94  2018-10-06 02:13:33.213988   470       NaN    NaN      NaN      0.0   
93  2018-10-06 02:13:33.213988   465       NaN    NaN      NaN      0.0   
92  2018-10-06 02:13:33.213988   460       NaN    NaN      NaN      0.0   
91  2018-10-06 02:13:33.213988   455       NaN    NaN      NaN      0.0   
286 2018-10-06 02:13:33.213988  1430       NaN    NaN      NaN      2.0   
287 2018-10-06 02:13:33.213988  1435       NaN    NaN      NaN      2.0   

     cohort2  cohort3   weekday  
98       3.0      9.0  Saturday  
97       3.0      9.0  Saturday  
96       2.0      8.0  Saturday  
95       2.0      8.0  Saturday  
94       2.0      8.0  Saturday  
93       2.0      8.0  Saturday  
92       2.0      8.0  Saturday  
91       2.0      8.0  Saturday  
286      8.0     26.0  Saturday  
287      8.0     26.0  Saturday

In [618]:
df = df.sort_values('Quality', ascending=False)
df.head(10)

Date  Goal  Complete     Score  Quality  cohort1  \
0 2018-10-06 00:00:00.000000     5       1.0  6.898648      0.5     -0.0   
1 2018-10-06 02:13:33.213988     5       NaN       NaN      NaN     -0.0   
2 2018-10-06 02:13:33.213988    10       NaN       NaN      NaN      0.0   
3 2018-10-06 02:13:33.213988    15       NaN       NaN      NaN      0.0   
4 2018-10-06 02:13:33.213988    20       NaN       NaN      NaN      0.0   
5 2018-10-06 02:13:33.213988    25       NaN       NaN      NaN      0.0   
6 2018-10-06 02:13:33.213988    30       NaN       NaN      NaN      0.0   
7 2018-10-06 02:13:33.213988    35       NaN       NaN      NaN      0.0   
8 2018-10-06 02:13:33.213988    40       NaN       NaN      NaN      0.0   
9 2018-10-06 02:13:33.213988    45       NaN       NaN      NaN      0.0   

   cohort2  cohort3   weekday  
0     -0.0     -0.0  Saturday  
1     -0.0     -0.0  Saturday  
2      0.0      0.0  Saturday  
3      0.0      0.0  Saturday  
4      0.0      0.0  Saturday  
5      0.0      0.0  Saturday  
6      0.0      0.0  Saturday  
7      0.0      0.0  Saturday  
8      0.0      0.0  Saturday  
9      0.0      0.0  Saturday

In [619]:
df_step1 = df_search(df,'cohort1')
df_step1

   cohort1  count  mean  var  total  donext
1      1.0      0   NaN  NaN      1   999.0
2      2.0      0   NaN  NaN      1   999.0
0     -0.0      1   0.5  NaN      1     0.5


Date  Goal  Complete  Score  Quality  cohort1  \
97  2018-10-06 02:13:33.213988   485       NaN    NaN      NaN      1.0   
98  2018-10-06 02:13:33.213988   490       NaN    NaN      NaN      1.0   
99  2018-10-06 02:13:33.213988   495       NaN    NaN      NaN      1.0   
100 2018-10-06 02:13:33.213988   500       NaN    NaN      NaN      1.0   
101 2018-10-06 02:13:33.213988   505       NaN    NaN      NaN      1.0   
102 2018-10-06 02:13:33.213988   510       NaN    NaN      NaN      1.0   
103 2018-10-06 02:13:33.213988   515       NaN    NaN      NaN      1.0   
104 2018-10-06 02:13:33.213988   520       NaN    NaN      NaN      1.0   
105 2018-10-06 02:13:33.213988   525       NaN    NaN      NaN      1.0   
106 2018-10-06 02:13:33.213988   530       NaN    NaN      NaN      1.0   
107 2018-10-06 02:13:33.213988   535       NaN    NaN      NaN      1.0   
108 2018-10-06 02:13:33.213988   540       NaN    NaN      NaN      1.0   
109 2018-10-06 02:13:33.213988   545       NaN    NaN      NaN      1.0   
110 2018-10-06 02:13:33.213988   550       NaN    NaN      NaN      1.0   
111 2018-10-06 02:13:33.213988   555       NaN    NaN      NaN      1.0   
112 2018-10-06 02:13:33.213988   560       NaN    NaN      NaN      1.0   
113 2018-10-06 02:13:33.213988   565       NaN    NaN      NaN      1.0   
114 2018-10-06 02:13:33.213988   570       NaN    NaN      NaN      1.0   
115 2018-10-06 02:13:33.213988   575       NaN    NaN      NaN      1.0   
116 2018-10-06 02:13:33.213988   580       NaN    NaN      NaN      1.0   
117 2018-10-06 02:13:33.213988   585       NaN    NaN      NaN      1.0   
118 2018-10-06 02:13:33.213988   590       NaN    NaN      NaN      1.0   
119 2018-10-06 02:13:33.213988   595       NaN    NaN      NaN      1.0   
120 2018-10-06 02:13:33.213988   600       NaN    NaN      NaN      1.0   
121 2018-10-06 02:13:33.213988   605       NaN    NaN      NaN      1.0   
122 2018-10-06 02:13:33.213988   610       NaN    NaN      NaN      1.0   
123 2018-10-06 02:13:33.213988   615       NaN    NaN      NaN      1.0   
124 2018-10-06 02:13:33.213988   620       NaN    NaN      NaN      1.0   
125 2018-10-06 02:13:33.213988   625       NaN    NaN      NaN      1.0   
126 2018-10-06 02:13:33.213988   630       NaN    NaN      NaN      1.0   
..                         ...   ...       ...    ...      ...      ...   
163 2018-10-06 02:13:33.213988   815       NaN    NaN      NaN      1.0   
164 2018-10-06 02:13:33.213988   820       NaN    NaN      NaN      1.0   
165 2018-10-06 02:13:33.213988   825       NaN    NaN      NaN      1.0   
166 2018-10-06 02:13:33.213988   830       NaN    NaN      NaN      1.0   
167 2018-10-06 02:13:33.213988   835       NaN    NaN      NaN      1.0   
168 2018-10-06 02:13:33.213988   840       NaN    NaN      NaN      1.0   
169 2018-10-06 02:13:33.213988   845       NaN    NaN      NaN      1.0   
170 2018-10-06 02:13:33.213988   850       NaN    NaN      NaN      1.0   
171 2018-10-06 02:13:33.213988   855       NaN    NaN      NaN      1.0   
172 2018-10-06 02:13:33.213988   860       NaN    NaN      NaN      1.0   
173 2018-10-06 02:13:33.213988   865       NaN    NaN      NaN      1.0   
174 2018-10-06 02:13:33.213988   870       NaN    NaN      NaN      1.0   
175 2018-10-06 02:13:33.213988   875       NaN    NaN      NaN      1.0   
176 2018-10-06 02:13:33.213988   880       NaN    NaN      NaN      1.0   
177 2018-10-06 02:13:33.213988   885       NaN    NaN      NaN      1.0   
178 2018-10-06 02:13:33.213988   890       NaN    NaN      NaN      1.0   
179 2018-10-06 02:13:33.213988   895       NaN    NaN      NaN      1.0   
180 2018-10-06 02:13:33.213988   900       NaN    NaN      NaN      1.0   
181 2018-10-06 02:13:33.213988   905       NaN    NaN      NaN      1.0   
182 2018-10-06 02:13:33.213988   910       NaN    NaN      NaN      1.0   
183 2018-10-06 02:13:33.213988   915       NaN    NaN      NaN      1.0   
184 2018-10-06 02:13:33.213988   920       NaN    N

In [620]:
df_step2 = df_search(df_step1,'cohort2')
df_step2

   cohort2  count  mean  var  total  donext
0      3.0      0   NaN  NaN      0   999.0
1      4.0      0   NaN  NaN      0   999.0
2      5.0      0   NaN  NaN      0   999.0


Date  Goal  Complete  Score  Quality  cohort1  \
97  2018-10-06 02:13:33.213988   485       NaN    NaN      NaN      1.0   
98  2018-10-06 02:13:33.213988   490       NaN    NaN      NaN      1.0   
99  2018-10-06 02:13:33.213988   495       NaN    NaN      NaN      1.0   
100 2018-10-06 02:13:33.213988   500       NaN    NaN      NaN      1.0   
101 2018-10-06 02:13:33.213988   505       NaN    NaN      NaN      1.0   
102 2018-10-06 02:13:33.213988   510       NaN    NaN      NaN      1.0   
103 2018-10-06 02:13:33.213988   515       NaN    NaN      NaN      1.0   
104 2018-10-06 02:13:33.213988   520       NaN    NaN      NaN      1.0   
105 2018-10-06 02:13:33.213988   525       NaN    NaN      NaN      1.0   
106 2018-10-06 02:13:33.213988   530       NaN    NaN      NaN      1.0   
107 2018-10-06 02:13:33.213988   535       NaN    NaN      NaN      1.0   
108 2018-10-06 02:13:33.213988   540       NaN    NaN      NaN      1.0   
109 2018-10-06 02:13:33.213988   545       NaN    NaN      NaN      1.0   
110 2018-10-06 02:13:33.213988   550       NaN    NaN      NaN      1.0   
111 2018-10-06 02:13:33.213988   555       NaN    NaN      NaN      1.0   
112 2018-10-06 02:13:33.213988   560       NaN    NaN      NaN      1.0   
113 2018-10-06 02:13:33.213988   565       NaN    NaN      NaN      1.0   
114 2018-10-06 02:13:33.213988   570       NaN    NaN      NaN      1.0   
115 2018-10-06 02:13:33.213988   575       NaN    NaN      NaN      1.0   
116 2018-10-06 02:13:33.213988   580       NaN    NaN      NaN      1.0   
117 2018-10-06 02:13:33.213988   585       NaN    NaN      NaN      1.0   
118 2018-10-06 02:13:33.213988   590       NaN    NaN      NaN      1.0   
119 2018-10-06 02:13:33.213988   595       NaN    NaN      NaN      1.0   
120 2018-10-06 02:13:33.213988   600       NaN    NaN      NaN      1.0   
121 2018-10-06 02:13:33.213988   605       NaN    NaN      NaN      1.0   
122 2018-10-06 02:13:33.213988   610       NaN    NaN      NaN      1.0   
123 2018-10-06 02:13:33.213988   615       NaN    NaN      NaN      1.0   
124 2018-10-06 02:13:33.213988   620       NaN    NaN      NaN      1.0   
125 2018-10-06 02:13:33.213988   625       NaN    NaN      NaN      1.0   
126 2018-10-06 02:13:33.213988   630       NaN    NaN      NaN      1.0   
127 2018-10-06 02:13:33.213988   635       NaN    NaN      NaN      1.0   
128 2018-10-06 02:13:33.213988   640       NaN    NaN      NaN      1.0   

     cohort2  cohort3   weekday  
97       3.0      9.0  Saturday  
98       3.0      9.0  Saturday  
99       3.0      9.0  Saturday  
100      3.0      9.0  Saturday  
101      3.0      9.0  Saturday  
102      3.0      9.0  Saturday  
103      3.0      9.0  Saturday  
104      3.0      9.0  Saturday  
105      3.0      9.0  Saturday  
106      3.0      9.0  Saturday  
107      3.0      9.0  Saturday  
108      3.0     10.0  Saturday  
109      3.0     10.0  Saturday  
110      3.0     10.0  Saturday  
111      3.0     10.0  Saturday  
112      3.0     10.0  Saturday  
113      3.0     10.0  Saturday  
114      3.0     10.0  Saturday  
115      3.0     10.0  Saturday  
116      3.0     10.0  Saturday  
117      3.0     10.0  Saturday  
118      3.0     11.0  Saturday  
119      3.0     11.0  Saturday  
120      3.0     11.0  Saturday  
121      3.0     11.0  Saturday  
122      3.0     11.0  Saturday  
123      3.0     11.0  Saturday  
124      3.0     11.0  Saturday  
125      3.0     11.0  Saturday  
126      3.0     11.0  Saturday  
127      3.0     11.0  Saturday  
128      3.0     11.0  Saturday

In [621]:
df_step3 = df_search(df_step2,'cohort3')
df_step3

   cohort3  count  mean  var  total  donext
0      9.0      0   NaN  NaN      0   999.0
1     10.0      0   NaN  NaN      0   999.0
2     11.0      0   NaN  NaN      0   999.0


Date  Goal  Complete  Score  Quality  cohort1  \
97  2018-10-06 02:13:33.213988   485       NaN    NaN      NaN      1.0   
98  2018-10-06 02:13:33.213988   490       NaN    NaN      NaN      1.0   
99  2018-10-06 02:13:33.213988   495       NaN    NaN      NaN      1.0   
100 2018-10-06 02:13:33.213988   500       NaN    NaN      NaN      1.0   
101 2018-10-06 02:13:33.213988   505       NaN    NaN      NaN      1.0   
102 2018-10-06 02:13:33.213988   510       NaN    NaN      NaN      1.0   
103 2018-10-06 02:13:33.213988   515       NaN    NaN      NaN      1.0   
104 2018-10-06 02:13:33.213988   520       NaN    NaN      NaN      1.0   
105 2018-10-06 02:13:33.213988   525       NaN    NaN      NaN      1.0   
106 2018-10-06 02:13:33.213988   530       NaN    NaN      NaN      1.0   
107 2018-10-06 02:13:33.213988   535       NaN    NaN      NaN      1.0   

     cohort2  cohort3   weekday  
97       3.0      9.0  Saturday  
98       3.0      9.0  Saturday  
99       3.0      9.0  Saturday  
100      3.0      9.0  Saturday  
101      3.0      9.0  Saturday  
102      3.0      9.0  Saturday  
103      3.0      9.0  Saturday  
104      3.0      9.0  Saturday  
105      3.0      9.0  Saturday  
106      3.0      9.0  Saturday  
107      3.0      9.0  Saturday

In [622]:
df = df_bandit_class(df_step3, 'Goal')
df['CanStart'] = df.apply(set_start, axis=1).dt.strftime('%I:%M %p')
df

Goal  count  mean  var  total  donext  CanStart
0    485      0   NaN  NaN      0   999.0  08:05 AM
1    490      0   NaN  NaN      0   999.0  08:10 AM
2    495      0   NaN  NaN      0   999.0  08:15 AM
3    500      0   NaN  NaN      0   999.0  08:20 AM
4    505      0   NaN  NaN      0   999.0  08:25 AM
5    510      0   NaN  NaN      0   999.0  08:30 AM
6    515      0   NaN  NaN      0   999.0  08:35 AM
7    520      0   NaN  NaN      0   999.0  08:40 AM
8    525      0   NaN  NaN      0   999.0  08:45 AM
9    530      0   NaN  NaN      0   999.0  08:50 AM
10   535      0   NaN  NaN      0   999.0  08:55 AM